# Обучение модели для wildfire.ai (https://wildfire.sberbank.ai/competition)

Оставляю почти без изменений весь код, чтобы случайно что-то не поменять, поэтому тут далеко не все красиво - есть копипаста, не все переменные адекватно названы, лишние print'ы и т.п....

Весь конкурс я работал на ноуте (Interl Core I5-8265U, 4 cores, 8 гб RAM), поэтому запуститься должно у всех

Докер-образ использовал sberbank/mchs-python, который подготовили организаторы, версии библиотек можно подтянуть оттуда для воспроизводимости

Поход по лидерборду был как-то так:
* 617 фич (брал только из ncep + пожары за предыдущее время, без кластеризации и городов) - ***0.9187 паблик***
* 611 фичей (убрал какие-то старые, добавил что-то, точно не помню) - ***0.9235 паблик***
* 677 фичей (добавил кластеризацию, день в году и год убрал из категориальных) - ***0.9253 паблик***;
* 228 фичей (нагенерил дополнительных фич (все которые в этом ноутбуке, кроме городов и лесов, полей, заповедников), отобрал по permutaion importance) - ***0.9289 паблик***
* 236 фич, добавил города, месяц убрал из категориальнфх фич, немного поменял параметры lightgbm, добавил леса) - ***0.9297 паблик***;
* 242 фич, добавил еще поля и заповедники - ***0.9299 паблик***
* 215 фич (дополнительно убрал некоторые по permutation importance) - ***0.9303 паблик***
* 214 фич (убрал месяц из фич) - ***0.9305 паблик***

на привате выиграла последняя модель


In [23]:
import pandas as pd
import numpy as np

import math
import time
import itertools
import xarray
import datetime
from dateutil.relativedelta import relativedelta
import re

from matplotlib import pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KDTree
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('best_solution')
from train_features import model_features # финальные фичи, отобранные при помощи permutation importance

In [2]:
df_train = pd.read_csv('wildfires_train.csv')
df_train['date'] = pd.to_datetime(df_train['date'])
df_train['timestamp'] = df_train['date'].apply(lambda x: time.mktime(x.timetuple()))
df_train.set_index('fire_id',inplace=True)

In [3]:
def get_day_of_year(date):
    '''
        хотел на самом деле считать по-другому и в конце считал по другому, но в итоге попала эта версия
        день в году:
          1) если год не високосный, то просто день в году
          2) если високосный - то если это 29 февраля, то полагаем день = 366, 
          если дата >= 1 марта, отнимаем 1 день (чтобы дни совпадали для високосного и невисокосных)
    '''
    doy = date.timetuple().tm_yday
    if date.year in (2012,2016,2020):
        if date >= datetime.date(date.year,3,1):
            doy -= 1
        elif date == datetime.date(date.year,2,29):
            doy = 366
    return doy

def calc_bearing(lon1,lat1,lon2,lat2):
    '''
        угол между двумя точками
    '''
    delta_lon = lon2 - lon1
    X = np.cos(math.radians(lat2)) * np.sin(math.radians(delta_lon))
    Y = np.cos(math.radians(lat1)) * np.sin(math.radians(lat2)) - np.sin(math.radians(lat1)) * np.cos(math.radians(lat2)) * np.cos(math.radians(delta_lon))
    bearing = math.atan2(X,Y)
    return bearing * 180/math.pi

def calc_distance(lon1,lat1,lon2,lat2):
    '''  расстояние между двумя точками '''
    radius = 6371302  # m

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c
    return d

# Ниже основной класс для генерации фич.

По умолчанию взял который были на сервере - uwind,air,rhum - они в итоговой модели только и остались исторические.

Еще взял vwind, но их брал только за определенную дату.

Из других открытых источников:
* города (брал отсюда https://gist.github.com/nalgeon/5307af065ff0e3bc97927c832fabe26b);
* поля, леса, заповедники взял из решения команды Gardeners https://github.com/VovaForbes/WildFire 

# Основные идеи
<ol>
    <li>Так как размер шага в данных ncep очень большой (2.5x2.5 градуса), то вряд ли разные статистики (типа скользящих средний и т.д.) много накинут в качестве, так как реальные значения в конкретной точке могут быть совсем другие. Но если рассматривать квадраты с центром в узлах сетки ncep, то мы получаем много квадратных полигонов, на которые разбита Россия. В этом случае каждый полигон, помимо координат, можно охарактеризовать еще набором других каких-либо параметров - например, количество пожаров в полигоне, ncep данные за конкретную дату (я в итоге брал за 09.2019, 10.2019, 11.2019) и т.п. Главная идея была в том, что координаты - это две переменные, чтобы найти похожие несколько полигонов по пожарам, модели нужно сделать много итераций, и в этом случае мы помогаем модели этими дополнительными фичами.
    Если я убирал эти фичи из рассмотрения, скор на валидации очень прилично падал (на паблике даже не проверял, больше доверял CV).</li>
    <li> Если проверять статистику по пожарам, то из года в год она все-таки прилично меняется, хоть и корреляция какая-то есть (например, если брать в качестве предикта KNN за последние три года в диапазоне +- 21 день от текущего дня, auc получатся около 0.73). Поэтому я специально не стал брать разные таргет энкодинги. Поэтому из исторических я только взял такие фичи:
        <ol>
        <li>координаты и расстояние до ближайшего пожара +- 14 дней за последние 3 года (не считая текущего года) и общее количество найденных пожаров за этот период</li>
            <li> общее количество пожаров за 3 года +- 7 дней от текущей даты</li>
            <li>координаты до ближайшего пожара в тот же день на расстоянии 10, 25 км, расстояние до этого пожара (если такой есть) и угол между двумя точками</li>
        </ol></li> 
    <li>Зато достаточно важная информация по размеру кластеров, в которых были пожары. Данные я только учитывал только за предыдущий год (очень важно это было учесть при кросс-валидации, эту фичу каждый раз приходилось делать заново), в итоге вышли такие фичи (алгоритм кластеризации DBSCAN):
    <ol>
        <li>для eps=0.035 (параметр DBSCAN) - количество пожаров в кластере, центр кластера (просто среднее по всем координатам), площадь внешнего прямоугольника, охватывающего весь кластер (вычисляется просто - ищутся в кластере макс, мин координаты, по ним площадь); </li>
         <li>для eps=0.15 (параметр DBSCAN) - количество пожаров в кластере, центр кластера (просто среднее по всем координатам), площадь внешнего прямоугольника, охватывающего весь кластер (вычисляется просто - ищутся в кластере макс, мин координаты, по ним площадь); </li>    
        <li> отношение площадей прямоугольников и количества в кластере для eps=0.035 и eps=0.15</li>
        </ol>
    <li> на основе ncep еще дополнительно формировал модуль скорости ветра wind_module ((uwind**2+vwind**2)**0.5) и направление ветра wind_direction (как считается можно посмотреть в wind_virection). Фичи: 
    <ol>
        <li> значения в точке по uwind,air,rhum </li>
        <li> среднее значение и среднее гармоническое по 4 ближайшим точкам из ncep для uwind,air,rhum </li>
        <li> среднее взвешенное по расстоянию от точки до точек сетки по 4 ближайшим точкам из ncep для uwind,air,rhum </li>
        <li>разные скользящие средние (7,14,21,35 дней) по всем доступным уровням (base_levels) по uwind,air,rhum; </li>
        <li> с </li>
        <li> разница между текущим значением и скользящим средним за 14 дней по uwind,air,rhum </li>
        <li> значения в точке для uwind,vwind,wind_module,wind_direction,air,rhum за 01.11.2019 и за 01.09.2019</li>
        <li> значения в точке скользящего среднего за 21 день для uwind,vwind,wind_module,wind_direction,air,rhum за 01.11.2019 </li>
        <li> значения в точке скользящего среднего за 7 дней для uwind,vwind,wind_module,wind_direction,air,rhum за 01.10.2019 </li>
        <li> среднее значение и среднее гармоническое по 4 ближайшим точкам из ncep для uwind,vwind,wind_module,wind_direction,air,rhum за 01.09.2019 и 01.11.2019 </li>
         <li> разные разницы между значениями, полученными выше </li>       
    </ol>
    </li>
    <li> из дат я нагенерил несколько фич, но в итоговой модели осталась только две - год и день в году (он считается немного хитро, чтобы учитывать високосный год) </li>
    <li> расстояние до ближайшего города, население этого города и его координаты</li>
    <li> расстояние до ближайшего поля, леса, заповедника и координаты</li>
    
для вычисления расстояний везде использовал KDTree - очень удобно и достаточно быстро;

для кластеризации - DBSCAN (находит кластеры разной формы, для кластеризации по координатам фитится только eps, удобно)

итоговая модель - lightgbm (бустинг), параметры подкрутил вручную


In [9]:
class FeaturesMaker():
    def __init__(self):
        self.lat_min = 32.5
        self.lat_max = 75
        self.lon_min = 12.5
        self.lon_max = 182.5      
        self.base_levels = (1000,700,500,300,250,200,100,150,70,50)
        self.rhum_levels = (1000,850,700,500,400,300)
        #self.base_levels = (1000,700,50,100)
        self.ncep_data = None
        self.lat_lons = np.array([x for x in itertools.product(np.arange(self.lon_min,self.lon_max+1,2.5),\
                                    np.arange(self.lat_min,self.lat_max+1,2.5))])
        self.kd_tree = KDTree(self.lat_lons)
    
    def make_raw_features(self,df):
        start = time.time()
        print('Making raw features...')
        basic_features = ['latitude','longitude']
        features_df = df[['fire_id','latitude','longitude','date']].copy()
        features_df['number_of_week'] = features_df['date'].apply(lambda x: x.isocalendar()[1])
        features_df['day_of_year'] = features_df['date'].apply(get_day_of_year)
        features_df['month'] = features_df['date'].apply(lambda x: x.month)
        features_df['year'] = features_df['date'].apply(lambda x: x.year)
        print('Making raw features done! Time {:.2f}'.format(time.time()-start))
        return features_df.drop('date',axis=1).set_index('fire_id')
    
    def make_temp_rhum_features(self,df):
        start = time.time()
        print('Reading temperature-rhum-dataset...')
        self.read_temp_rhum_set()
        print("Reading dataset done! Time {:.2f}".format(time.time()-start))
        start = time.time()
        print('Making temperature-humidity features...')
        temp_rhum_features = self.extract_temp_rhum_features(df)
        print(pd.DataFrame(temp_rhum_features).shape)
        print('Making temperature-humidity features done! Time {:.2f}'.format(time.time()-start))
        return pd.DataFrame(temp_rhum_features).set_index('fire_id')
        
    def wind_direction(self,u,v,wind_module):
        try:
            wind_dir_trig_to = np.arctan2(u/wind_module,  v/wind_module)* 180/math.pi
        except:
            wind_dir_trig_to = 0    
        return wind_dir_trig_to
            
    def read_temp_rhum_set(self):        
        ncep_data = []
        for var in ('rhum','air'): #'uwnd','vwnd', 'rhum','trpp','srfpt'):
            print(var)
            for year in range(2012,2020,1):
                dataset_filename = '{}.{}.nc'.format(var, year)
                array = xarray.open_dataset(dataset_filename)
                array = array.where((array.lat >= self.lat_min) &
                      (array.lat <= self.lat_max) & 
                      (array.lon <= self.lon_max) & 
                      (array.lon >= self.lon_min) & 
                     (array.level.isin(self.base_levels)),drop=True)
                ncep_data.append(array)
            full_array = xarray.merge(ncep_data)
        self.ncep_data = full_array
        
    def extract_temp_rhum_features(self,df):   
        features_df = df[['fire_id','longitude','latitude','lat','lon','lat_near_2','lon_near_2','lat_near_3','lon_near_3','lat_near_4','lon_near_4',
                                  'time']].copy()
        features_df['bearing_from_nearest'] = features_df[['lon','lat','longitude','latitude']]\
                                   .apply(lambda x: calc_bearing(x['lon'],x['lat'],x['longitude'],x['latitude']),axis=1)
        features_df['distance_from_nearest'] = features_df[['lon','lat','longitude','latitude']]\
                                   .apply(lambda x: calc_distance(x['lon'],x['lat'],x['longitude'],x['latitude']),axis=1)
        
        for level in self.base_levels:
            point_df = self.ncep_data.sel(
                level=level,
            ).to_dataframe().reset_index().rename(columns={'air':'temperature_level{}'.format(level),
                                                          'rhum':'humidity_level{}'.format(level)})
            air = 'temperature_level{}'.format(level)
            rhum = 'humidity_level{}'.format(level)
            
            if level == self.base_levels[0]:
                print(point_df['time'].max(),point_df['time'].min())

            p21d = point_df.groupby(['lat','lon']).rolling(window=21,min_periods=1,on='time')[[air,rhum]].mean().reset_index().\
                  rename(columns={air:'p21d-t_level{}'.format(level),rhum:'p21d-h_level{}'.format(level)})
            
            p35d = point_df.groupby(['lat','lon']).rolling(window=35,min_periods=1,on='time')[[air,rhum]].mean().reset_index().\
                  rename(columns={air:'p35d-t_level{}'.format(level),rhum:'p35d-h_level{}'.format(level)})
            p35d['time'] = p35d['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            p7d = point_df.groupby(['lat','lon']).rolling(window=7,min_periods=1,on='time')[[air,rhum]].mean().reset_index().\
                  rename(columns={air:'p7d-t_level{}'.format(level),rhum:'p7d-h_level{}'.format(level)})
            
            p14d = point_df.groupby(['lat','lon']).rolling(window=14,min_periods=1,on='time')[[air,rhum]].mean().reset_index().\
                  rename(columns={air:'p14d-t_level{}'.format(level),rhum:'p14d-h_level{}'.format(level)})
            p14d['time'] = p14d['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            #point_df_diff = point_df.groupby(['lat','lon'])[[air,rhum]].diff().fillna(0).reset_index().\
            #             rename(columns={air:'t_prev_diff_level{}'.format(level),rhum:'h_prev_diff_level{}'.format(level)})
            #point_df_diff = point_df_diff.merge(point_df[['time','lat','lon']],left_index=True,right_index=True)
            #point_df_diff['time'] = point_df_diff['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            lat_lon_df = point_df[point_df['time'] >= datetime.date(2019,11,1)].copy()
            lat_lon_df.drop_duplicates(subset=['lat','lon'],keep='first',inplace=True)
            lat_lon_df.rename(columns={'temperature_level{}'.format(level):'first_temperature_level{}'.format(level),
                                      'humidity_level{}'.format(level):'first_humidity_level{}'.format(level)},inplace=True)
            lat_lon_df['time'] = lat_lon_df['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            lat_lon_df_2 = point_df[point_df['time'] >= datetime.date(2019,9,1)].copy()
            lat_lon_df_2.drop_duplicates(subset=['lat','lon'],keep='first',inplace=True)
            lat_lon_df_2.rename(columns={'temperature_level{}'.format(level):'first_20190901_temperature_level{}'.format(level),
                                      'humidity_level{}'.format(level):'first_20190901_humidity_level{}'.format(level)},inplace=True)
            lat_lon_df_2['time'] = lat_lon_df_2['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            lat_lon_df_21 = p21d[p21d['time'] >= datetime.date(2019,11,1)].copy()
            lat_lon_df_21.drop_duplicates(subset=['lat','lon'],keep='first',inplace=True)
            lat_lon_df_21.rename(columns={'p21d-t_level{}'.format(level):'first_21d_temperature_level{}'.format(level),
                                      'p21d-h_level{}'.format(level):'first_21d_humidity_level{}'.format(level)},inplace=True)
            lat_lon_df_21['time'] = lat_lon_df_21['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            lat_lon_df_7 = p7d[p7d['time'] >= datetime.date(2019,10,1)].copy()
            lat_lon_df_7.drop_duplicates(subset=['lat','lon'],keep='first',inplace=True)
            lat_lon_df_7.rename(columns={'p7d-t_level{}'.format(level):'first_7d_temperature_level{}'.format(level),
                                      'p7d-h_level{}'.format(level):'first_7d_humidity_level{}'.format(level)},inplace=True)
            lat_lon_df_7['time'] = lat_lon_df_7['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            p21d['time'] = p21d['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            p7d['time'] = p7d['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            point_df['time'] = point_df['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            features_df = features_df.merge(p14d[['lat','lon','time','p14d-t_level{}'.format(level),'p14d-h_level{}'.format(level)]],
                                on=['lat','lon','time'])
            features_df = features_df.merge(p7d[['lat','lon','time','p7d-t_level{}'.format(level),'p7d-h_level{}'.format(level)]],
                                on=['lat','lon','time'])
            features_df = features_df.merge(p21d[['lat','lon','time','p21d-t_level{}'.format(level),'p21d-h_level{}'.format(level)]],
                                on=['lat','lon','time'])
            features_df = features_df.merge(p35d[['lat','lon','time','p35d-t_level{}'.format(level),'p35d-h_level{}'.format(level)]],
                                on=['lat','lon','time'])
            features_df = features_df.merge(point_df[['lat','lon','time','temperature_level{}'.format(level),'humidity_level{}'.format(level)]],
                                on=['lat','lon','time'])
            #features_df = features_df.merge(point_df_diff[['lat','lon','time','t_prev_diff_level{}'.format(level),
            #                            'h_prev_diff_level{}'.format(level)]],on=['lat','lon','time'])
            
            features_df = features_df.merge(lat_lon_df_2[['lat','lon','first_20190901_temperature_level{}'.format(level),
                                            'first_20190901_humidity_level{}'.format(level)]],on=['lat','lon'])
            features_df = features_df.merge(lat_lon_df[['lat','lon','first_temperature_level{}'.format(level),
                                            'first_humidity_level{}'.format(level)]],on=['lat','lon'])
            features_df = features_df.merge(lat_lon_df_21[['lat','lon','first_21d_temperature_level{}'.format(level),
                                            'first_21d_humidity_level{}'.format(level)]],on=['lat','lon'])
            features_df = features_df.merge(lat_lon_df_7[['lat','lon','first_7d_temperature_level{}'.format(level),
                                            'first_7d_humidity_level{}'.format(level)]],on=['lat','lon'])
            features_df['p14d_diff-t_level{}'.format(level)] = (features_df['temperature_level{}'.format(level)] - \
                                               features_df['p14d-t_level{}'.format(level)])
            features_df['p14d_diff-h_level{}'.format(level)] = (features_df['humidity_level{}'.format(level)] - \
                                               features_df['p14d-h_level{}'.format(level)])
            #features_df['p35d_diff-t_level{}'.format(level)] = (features_df['temperature_level{}'.format(level)] - \
             #                                  features_df['p35d-t_level{}'.format(level)])
            #features_df['p35d_diff-h_level{}'.format(level)] = (features_df['humidity_level{}'.format(level)] - \
             #                                  features_df['p35d-h_level{}'.format(level)])
            
            for nearest in [2,3,4]:
                features_df = features_df.merge(point_df[['lat','lon','time','temperature_level{}'.format(level),'humidity_level{}'.format(level)]].\
                                              rename(columns={'temperature_level{}'.format(level):'temperature_near_{}_level_{}'.format(nearest,level),
                                                              'humidity_level{}'.format(level):'humidity_near_{}_level_{}'.format(nearest,level),
                                                             'lat':'lat_near_{}'.format(nearest),
                                                             'lon':'lon_near_{}'.format(nearest)}),\
                                on=['lat_near_{}'.format(nearest),'lon_near_{}'.format(nearest),'time'])
                
                
                features_df = features_df.merge(lat_lon_df[['lat','lon','first_temperature_level{}'.format(level),
                                                          'first_humidity_level{}'.format(level)]].\
                    rename(columns={'first_temperature_level{}'.format(level):'first_temperature_near_{}_level_{}'.format(nearest,level),
                    'first_humidity_level{}'.format(level):'first_humidity_near_{}_level_{}'.format(nearest,level),
                    'lat':'lat_near_{}'.format(nearest),
                    'lon':'lon_near_{}'.format(nearest)}),\
                                on=['lat_near_{}'.format(nearest),'lon_near_{}'.format(nearest)]) 
                features_df['distance_{}'.format(nearest)] = features_df[['lon_near_{}'.format(nearest),'lat_near_{}'.format(nearest),'longitude','latitude']]\
                      .apply(lambda x: calc_distance(x['lon_near_{}'.format(nearest)],x['lat_near_{}'.format(nearest)],x['longitude'],x['latitude']),axis=1)
            
                
            
            features_df['harmonic_mean-t_level{}'.format(level)] = (4/((1/features_df['temperature_level{}'.format(level)]) +\
                                                           (1/features_df['temperature_near_2_level_{}'.format(level)]) +\
                                                           (1/features_df['temperature_near_3_level_{}'.format(level)]) +\
                                                           (1/features_df['temperature_near_4_level_{}'.format(level)])))
            
            features_df['harmonic_mean-h_level{}'.format(level)] = (4/((1/features_df['humidity_level{}'.format(level)]) +\
                                                           (1/features_df['humidity_near_2_level_{}'.format(level)]) +\
                                                           (1/features_df['humidity_near_3_level_{}'.format(level)]) +\
                                                           (1/features_df['humidity_near_4_level_{}'.format(level)])))

            features_df['dist_weighted_mean-t_level{}'.format(level)] = (features_df['temperature_level{}'.format(level)] *\
                                                           features_df['distance_from_nearest'] +\
                                                           features_df['temperature_near_2_level_{}'.format(level)] *\
                                                           features_df['distance_2'] +\
                                                           features_df['temperature_near_3_level_{}'.format(level)] *\
                                                           features_df['distance_3']+\
                                                           features_df['temperature_near_4_level_{}'.format(level)] *\
                                                           features_df['distance_4'])/(features_df['distance_from_nearest'] + \
                                                    features_df['distance_2']+features_df['distance_3']+features_df['distance_4'])
            
            features_df['dist_weighted_mean-h_level{}'.format(level)] = (features_df['humidity_level{}'.format(level)] *\
                                                           features_df['distance_from_nearest'] +\
                                                           features_df['humidity_near_2_level_{}'.format(level)] *\
                                                           features_df['distance_2'] +\
                                                           features_df['humidity_near_3_level_{}'.format(level)] *\
                                                           features_df['distance_3']+\
                                                           features_df['humidity_near_4_level_{}'.format(level)] *\
                                                           features_df['distance_4'])/(features_df['distance_from_nearest'] + \
                                                    features_df['distance_2']+features_df['distance_3']+features_df['distance_4'])
            
            features_df['mean-t_level{}'.format(level)] = (features_df['temperature_level{}'.format(level)] +\
                                                           features_df['temperature_near_2_level_{}'.format(level)] +\
                                                           features_df['temperature_near_3_level_{}'.format(level)] +\
                                                           features_df['temperature_near_4_level_{}'.format(level)])/4
            
            features_df['mean-h_level{}'.format(level)] = (features_df['humidity_level{}'.format(level)] +\
                                                           features_df['humidity_near_2_level_{}'.format(level)] +\
                                                           features_df['humidity_near_3_level_{}'.format(level)] +\
                                                           features_df['humidity_near_4_level_{}'.format(level)])/4
            
            
            features_df['first_mean-t_level{}'.format(level)] = (features_df['first_temperature_level{}'.format(level)] +\
                                                           features_df['first_temperature_near_2_level_{}'.format(level)] +\
                                                           features_df['first_temperature_near_3_level_{}'.format(level)] +\
                                                           features_df['first_temperature_near_4_level_{}'.format(level)])/4
            
            features_df['first_mean-h_level{}'.format(level)] = (features_df['first_humidity_level{}'.format(level)] +\
                                                           features_df['first_humidity_near_2_level_{}'.format(level)] +\
                                                           features_df['first_humidity_near_3_level_{}'.format(level)] +\
                                                           features_df['first_humidity_near_4_level_{}'.format(level)])/4
            
            features_df['dist_weighted_first_mean-t_level{}'.format(level)] = (features_df['first_temperature_level{}'.format(level)] *\
                                                           features_df['distance_from_nearest'] +\
                                                           features_df['first_temperature_near_2_level_{}'.format(level)] *\
                                                           features_df['distance_2'] +\
                                                           features_df['first_temperature_near_3_level_{}'.format(level)] *\
                                                           features_df['distance_3'] +\
                                                           features_df['first_temperature_near_4_level_{}'.format(level)] *\
                                                           features_df['distance_4'])/(features_df['distance_from_nearest'] + \
                                                    features_df['distance_2']+features_df['distance_3']+features_df['distance_4'])
            
            features_df['dist_weighted_irst_mean-h_level{}'.format(level)] = (features_df['first_humidity_level{}'.format(level)] *\
                                                           features_df['distance_from_nearest'] +\
                                                           features_df['first_humidity_near_2_level_{}'.format(level)] *\
                                                           features_df['distance_2'] +\
                                                           features_df['first_humidity_near_3_level_{}'.format(level)] *\
                                                           features_df['distance_3'] +\
                                                           features_df['first_humidity_near_4_level_{}'.format(level)] *\
                                                           features_df['distance_4'])/(features_df['distance_from_nearest'] + \
                                                    features_df['distance_2']+features_df['distance_3']+features_df['distance_4'])
            
            features_df['diff_mean-t_level{}'.format(level)] = features_df['temperature_level{}'.format(level)] - features_df['mean-t_level{}'.format(level)]
            features_df['diff_first_mean-t_level{}'.format(level)] = features_df['temperature_level{}'.format(level)] - features_df['first_mean-t_level{}'.format(level)]
            features_df['diff_first_mean-first_t_level{}'.format(level)] = features_df['first_temperature_level{}'.format(level)] - features_df['first_mean-t_level{}'.format(level)]
            features_df['diff_first-t_level{}'.format(level)] = features_df['temperature_level{}'.format(level)] - features_df['first_temperature_level{}'.format(level)]
            features_df['diff_mean-h_level{}'.format(level)] = features_df['humidity_level{}'.format(level)] - features_df['mean-h_level{}'.format(level)]
            features_df['diff_first_mean-h_level{}'.format(level)] = features_df['humidity_level{}'.format(level)] - features_df['first_mean-h_level{}'.format(level)]
            features_df['diff_first-h_level{}'.format(level)] = features_df['humidity_level{}'.format(level)] - features_df['first_humidity_level{}'.format(level)]
            features_df['diff_first_mean-first_h_level{}'.format(level)] = features_df['first_humidity_level{}'.format(level)] - features_df['first_mean-h_level{}'.format(level)]
            #features_df['diff_first-h-t_level{}'.format(level)] = features_df['diff_first-h_level{}'.format(level)] * features_df['diff_first-t_level{}'.format(level)]
            #features_df['h-t_level{}'.format(level)] = features_df['temperature_level{}'.format(level)] * features_df['humidity_level{}'.format(level)]
        for level_1,level_2 in [(1000,700),(100,50),(1000,50),(1000,300),(70,50),(150,50)]:
            features_df['temperature_level_{}_{}_diff'.format(level_1,level_2)] = features_df['temperature_level{}'.format(level_1)] -  features_df['temperature_level{}'.format(level_2)]
            features_df['p14d-t_level_{}_{}_diff'.format(level_1,level_2)] = features_df['p14d-t_level{}'.format(level_1)] -  features_df['p14d-t_level{}'.format(level_2)]
            features_df['first_mean-t_level_{}_{}_diff'.format(level_1,level_2)] = features_df['first_mean-t_level{}'.format(level_1)] -  features_df['first_mean-t_level{}'.format(level_2)]
            features_df['first-t_level_{}_{}_diff'.format(level_1,level_2)] = features_df['first_temperature_level{}'.format(level_1)] -  features_df['first_temperature_level{}'.format(level_2)]
        for level_1,level_2 in [(1000,700),(500,300),(1000,300),(700,300)]:
            features_df['humidity_level_{}_{}_diff'.format(level_1,level_2)] = features_df['humidity_level{}'.format(level_1)] -  features_df['humidity_level{}'.format(level_2)]
            features_df['p14d-h_level_{}_{}_diff'.format(level_1,level_2)] = features_df['p14d-h_level{}'.format(level_1)] -  features_df['p14d-h_level{}'.format(level_2)]
            features_df['first_mean-h_level_{}_{}_diff'.format(level_1,level_2)] = features_df['first_mean-h_level{}'.format(level_1)] -  features_df['first_mean-h_level{}'.format(level_2)]
            features_df['first-h_level_{}_{}_diff'.format(level_1,level_2)] = features_df['first_humidity_level{}'.format(level_1)] -  features_df['first_humidity_level{}'.format(level_2)]
        return features_df.drop(['lat_near_2','lon_near_2','lat_near_3','lon_near_3','lat_near_4','lon_near_4',
                                 'time','latitude','longitude',
                            'temperature_near_2_level_{}'.format(level),'temperature_near_3_level_{}'.format(level),
                                'temperature_near_4_level_{}'.format(level),'humidity_near_3_level_{}'.format(level),
                                'humidity_near_3_level_{}'.format(level),'humidity_near_3_level_{}'.format(level),
                                'first_temperature_near_2_level_{}'.format(level),'first_temperature_near_3_level_{}'.format(level),
                                'first_temperature_near_4_level_{}'.format(level),'first_humidity_near_3_level_{}'.format(level),
                                'first_humidity_near_3_level_{}'.format(level),'first_humidity_near_3_level_{}'.format(level)],
                                 axis=1,errors='ignore')   
                       
    
    def extract_winds_features(self,df):   
        features_df = df[['fire_id','longitude','latitude','lat','lon','lat_near_2','lon_near_2','lat_near_3','lon_near_3','lat_near_4','lon_near_4',
                                  'time']].copy()
        
        for level in self.base_levels:
            point_df = self.ncep_data.sel(
                level=level,
            ).to_dataframe().reset_index().rename(columns={'uwnd':'uwind_level{}'.format(level),
                                                          'vwnd':'vwind_level{}'.format(level)})
            uwnd = 'uwind_level{}'.format(level)
            vwnd = 'vwind_level{}'.format(level)
            
            if level == self.base_levels[0]:
                print(point_df['time'].max(),point_df['time'].min())
            
            p21d = point_df.groupby(['lat','lon']).rolling(window=21,min_periods=1,on='time')[[uwnd,vwnd]].mean().reset_index().\
                  rename(columns={uwnd:'p21d-uw_level{}'.format(level),vwnd:'p21d-vw_level{}'.format(level)})
            
            p35d = point_df.groupby(['lat','lon']).rolling(window=35,min_periods=1,on='time')[[uwnd]].mean().reset_index().\
                  rename(columns={uwnd:'p35d-uw_level{}'.format(level)})
            p35d['time'] = p35d['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            #p50d = point_df.groupby(['lat','lon']).rolling(window=50,min_periods=1,on='time')[[uwnd]].mean().reset_index().\
            #      rename(columns={uwnd:'p50d-uw_level{}'.format(level)})
            #p50d['time'] = p50d['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            p7d = point_df.groupby(['lat','lon']).rolling(window=7,min_periods=1,on='time')[[uwnd,vwnd]].mean().reset_index().\
                  rename(columns={uwnd:'p7d-uw_level{}'.format(level),vwnd:'p7d-vw_level{}'.format(level)})
            
            p14d = point_df.groupby(['lat','lon']).rolling(window=14,min_periods=1,on='time')[[uwnd]].mean().reset_index().\
                  rename(columns={uwnd:'p14d-uw_level{}'.format(level)})
            p14d['time'] = p14d['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            point_df_diff = point_df.groupby(['lat','lon'])[[uwnd]].diff().fillna(0).reset_index().\
                         rename(columns={uwnd:'uw_prev_diff_level{}'.format(level)})
            point_df_diff = point_df_diff.merge(point_df[['time','lat','lon']],left_index=True,right_index=True)
            point_df_diff['time'] = point_df_diff['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            lat_lon_df = point_df[point_df['time'] >= datetime.date(2019,11,1)].copy()
            lat_lon_df.drop_duplicates(subset=['lat','lon'],keep='first',inplace=True)
            lat_lon_df.rename(columns={'uwind_level{}'.format(level):'first_uwind_level{}'.format(level),
                                      'vwind_level{}'.format(level):'first_vwind_level{}'.format(level)},inplace=True)
            lat_lon_df['time'] = lat_lon_df['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            lat_lon_df_2 = point_df[point_df['time'] >= datetime.date(2019,9,1)].copy()
            lat_lon_df_2.drop_duplicates(subset=['lat','lon'],keep='first',inplace=True)
            lat_lon_df_2.rename(columns={'uwind_level{}'.format(level):'first_20190901_uwind_level{}'.format(level),
                                      'vwind_level{}'.format(level):'first_20190901_vwind_level{}'.format(level)},inplace=True)
            lat_lon_df_2['time'] = lat_lon_df_2['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            lat_lon_df_21 = p21d[p21d['time'] >= datetime.date(2019,11,1)].copy()
            lat_lon_df_21.drop_duplicates(subset=['lat','lon'],keep='first',inplace=True)
            lat_lon_df_21.rename(columns={'p21d-uw_level{}'.format(level):'first_21d_uwind_level{}'.format(level),
                                      'p21d-vw_level{}'.format(level):'first_21d_vwind_level{}'.format(level)},inplace=True)
            lat_lon_df_21['time'] = lat_lon_df_21['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            lat_lon_df_7 = p7d[p7d['time'] >= datetime.date(2019,10,1)].copy()
            lat_lon_df_7.drop_duplicates(subset=['lat','lon'],keep='first',inplace=True)
            lat_lon_df_7.rename(columns={'p7d-uw_level{}'.format(level):'first_7d_uwind_level{}'.format(level),
                                      'p7d-vw_level{}'.format(level):'first_7d_vwind_level{}'.format(level)},inplace=True)
            lat_lon_df_7['time'] = lat_lon_df_7['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            
            point_df['time'] = point_df['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            p21d['time'] = p21d['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
            p7d['time'] = p7d['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
                

            features_df = features_df.merge(p14d[['lat','lon','time','p14d-uw_level{}'.format(level)]],
                                on=['lat','lon','time'])
            features_df = features_df.merge(p7d[['lat','lon','time','p7d-uw_level{}'.format(level)]],
                                on=['lat','lon','time'])
            features_df = features_df.merge(p21d[['lat','lon','time','p21d-uw_level{}'.format(level)]],
                                on=['lat','lon','time'])
            features_df = features_df.merge(p35d[['lat','lon','time','p35d-uw_level{}'.format(level)]],
                                on=['lat','lon','time'])
            features_df = features_df.merge(point_df[['lat','lon','time','uwind_level{}'.format(level)]],
                                on=['lat','lon','time'])
            #features_df = features_df.merge(point_df_diff[['lat','lon','time',
            #                        'uw_prev_diff_level{}'.format(level)]],on=['lat','lon','time'])
            
            features_df = features_df.merge(lat_lon_df[['lat','lon','first_uwind_level{}'.format(level),
                                                       'first_vwind_level{}'.format(level)]],
                                            on=['lat','lon'])
            
            features_df['p14d_diff-uw_level{}'.format(level)] = (features_df['uwind_level{}'.format(level)] - \
                                               features_df['p14d-uw_level{}'.format(level)])
            #features_df['p35d_diff-uw_level{}'.format(level)] = (features_df['uwind_level{}'.format(level)] - \
            #                                   features_df['p35d-uw_level{}'.format(level)])

            for nearest in [2,3,4]:
                features_df = features_df.merge(point_df[['lat','lon','time','uwind_level{}'.format(level)]].\
                                              rename(columns={'uwind_level{}'.format(level):'uwind_near_{}_level_{}'.format(nearest,level),
                                                             'lat':'lat_near_{}'.format(nearest),
                                                             'lon':'lon_near_{}'.format(nearest)}),
                                on=['lat_near_{}'.format(nearest),'lon_near_{}'.format(nearest),'time'])
                
                features_df = features_df.merge(lat_lon_df[['lat','lon','first_uwind_level{}'.format(level),
                                                           'first_vwind_level{}'.format(level)]].\
                    rename(columns={'first_uwind_level{}'.format(level):'first_uwind_near_{}_level_{}'.format(nearest,level),
                                    'first_vwind_level{}'.format(level):'first_vwind_near_{}_level_{}'.format(nearest,level),
                    'lat':'lat_near_{}'.format(nearest),
                    'lon':'lon_near_{}'.format(nearest)}),\
                                on=['lat_near_{}'.format(nearest),'lon_near_{}'.format(nearest)]) 

              
            
            features_df['harmonic_mean-uw_level{}'.format(level)] = (4/((1/features_df['uwind_level{}'.format(level)]) +\
                                                           (1/features_df['uwind_near_2_level_{}'.format(level)]) +\
                                                           (1/features_df['uwind_near_3_level_{}'.format(level)]) +\
                                                           (1/features_df['uwind_near_4_level_{}'.format(level)])))
            
            #features_df['harmonic_mean-vw_level{}'.format(level)] = (4/((1/features_df['vwind_level{}'.format(level)]) +\
            #                                               (1/features_df['vwind_near_2_level_{}'.format(level)]) +\
            #                                               (1/features_df['vwind_near_3_level_{}'.format(level)]) +\
            #                                               (1/features_df['vwind_near_4_level_{}'.format(level)])))
            
            features_df['mean-uw_level{}'.format(level)] = (features_df['uwind_level{}'.format(level)] +\
                                                           features_df['uwind_near_2_level_{}'.format(level)] +\
                                                           features_df['uwind_near_3_level_{}'.format(level)] +\
                                                           features_df['uwind_near_4_level_{}'.format(level)])/4
            
            #features_df['mean-vw_level{}'.format(level)] = (features_df['vwind_level{}'.format(level)] +\
            #                                               features_df['vwind_near_2_level_{}'.format(level)] +\
            #                                               features_df['vwind_near_3_level_{}'.format(level)] +\
            #                                               features_df['vwind_near_4_level_{}'.format(level)])/4
            
            features_df['first_mean-uw_level{}'.format(level)] = (features_df['first_uwind_level{}'.format(level)] +\
                                                           features_df['first_uwind_near_2_level_{}'.format(level)] +\
                                                           features_df['first_uwind_near_3_level_{}'.format(level)] +\
                                                           features_df['first_uwind_near_4_level_{}'.format(level)])/4
            features_df['first_mean-vw_level{}'.format(level)] = (features_df['first_vwind_level{}'.format(level)] +\
                                                           features_df['first_vwind_near_2_level_{}'.format(level)] +\
                                                           features_df['first_vwind_near_3_level_{}'.format(level)] +\
                                                           features_df['first_vwind_near_4_level_{}'.format(level)])/4
            
            features_df['first_wind_module_level{}'.format(level)] = (features_df['first_vwind_level{}'.format(level)] ** 2 + \
                                                               features_df['first_uwind_level{}'.format(level)] ** 2)**0.5
            features_df['first_wind_direction_level{}'.format(level)] = features_df[['first_uwind_level{}'.format(level),
                                                'first_vwind_level{}'.format(level),
                                                'first_wind_module_level{}'.format(level)]].\
                                                 apply(lambda x:self.wind_direction(x['first_uwind_level{}'.format(level)],
                                                                x['first_vwind_level{}'.format(level)],
                                                                        x['first_wind_module_level{}'.format(level)]),axis=1)
            
            features_df['diff_mean-uw_level{}'.format(level)] = features_df['uwind_level{}'.format(level)] - features_df['mean-uw_level{}'.format(level)]
            features_df['diff_first_mean-uw_level{}'.format(level)] = features_df['uwind_level{}'.format(level)] - features_df['first_mean-uw_level{}'.format(level)]
            features_df['diff_first-uw_level{}'.format(level)] = features_df['uwind_level{}'.format(level)] - features_df['first_uwind_level{}'.format(level)]
            features_df['diff_first_mean-first_uw_level{}'.format(level)] = features_df['first_uwind_level{}'.format(level)] - features_df['first_mean-uw_level{}'.format(level)]
            features_df['diff_first_mean-first_vw_level{}'.format(level)] = features_df['first_vwind_level{}'.format(level)] - features_df['first_mean-vw_level{}'.format(level)]
        for level_1,level_2 in [(1000,700),(100,50),(1000,50),(1000,300),(70,50),(150,50)]:   
            features_df['uwind_level_{}_{}_diff'.format(level_1,level_2)] = features_df['uwind_level{}'.format(level_1)] -  features_df['uwind_level{}'.format(level_2)]
            features_df['p14d-uw_level_{}_{}_diff'.format(level_1,level_2)] = features_df['p14d-uw_level{}'.format(level_1)] -  features_df['p14d-uw_level{}'.format(level_2)]
            features_df['first_mean-uw_level_{}_{}_diff'.format(level_1,level_2)] = features_df['first_mean-uw_level{}'.format(level_1)] -  features_df['first_mean-uw_level{}'.format(level_2)]
            features_df['first_mean-vw_level_{}_{}_diff'.format(level_1,level_2)] = features_df['first_mean-vw_level{}'.format(level_1)] -  features_df['first_mean-vw_level{}'.format(level_2)]
            features_df['first-uw_level_{}_{}_diff'.format(level_1,level_2)] = features_df['first_uwind_level{}'.format(level_1)] -  features_df['first_uwind_level{}'.format(level_2)]
            features_df['first-vw_level_{}_{}_diff'.format(level_1,level_2)] = features_df['first_vwind_level{}'.format(level_1)] -  features_df['first_vwind_level{}'.format(level_2)]
            #features_df['first_mean-wind_module_level_{}_{}_diff'.format(level_1,level_2)] = features_df['first_wind_module_level{}'.format(level_1)] -  features_df['first_wind_module_level{}'.format(level_2)]
        
        return features_df.drop(['lat','lon','lat_near_2','lon_near_2','lat_near_3','lon_near_3','lat_near_4','lon_near_4','time',
                                'bearing_from_nearest','latitude','longitude',
                                'uwind_near_2_level_{}'.format(level),'uwind_near_3_level_{}'.format(level),
                                'uwind_near_4_level_{}'.format(level),'vwind_near_2_level_{}'.format(level),
                                'vwind_near_3_level_{}'.format(level),'vwind_near_4_level_{}'.format(level),
                                'first_uwind_near_2_level_{}'.format(level),'first_uwind_near_3_level_{}'.format(level),
                                'first_uwind_near_4_level_{}'.format(level),'first_vwind_near_2_level_{}'.format(level),
                                'first_vwind_near_3_level_{}'.format(level),'first_vwind_near_4_level_{}'.format(level)],
                                axis=1,errors='ignore')
    
    def make_winds_features(self,df):
        start = time.time()
        print('Reading winds dataset...')
        self.read_winds_set()
        print("Reading dataset done! Time {:.2f}".format(time.time()-start))
        start = time.time()
        print('Making wind features...')
        winds_features = self.extract_winds_features(df)
        print('Making wind features done! Time {:.2f}'.format(time.time()-start))
        print(pd.DataFrame(winds_features).shape)
        return pd.DataFrame(winds_features).set_index('fire_id')
    
    def read_winds_set(self):       
        ncep_data = []
        for var in ['uwnd','vwnd']:
            print(var)
            for year in range(2012,2020,1):
                dataset_filename = '{}.{}.nc'.format(var, year)
                array = xarray.open_dataset(dataset_filename)
                array = array.where((array.lat >= self.lat_min) &
                      (array.lat <= self.lat_max) & 
                      (array.lon <= self.lon_max) & 
                      (array.lon >= self.lon_min) & 
                     (array.level.isin(self.base_levels)),drop=True)
                ncep_data.append(array)
            full_array = xarray.merge(ncep_data)
        self.ncep_data = full_array
        
    def make_all_features(self,df):
        if df['date'].dtype== 'O':
            df['date'] = pd.to_datetime(df['date'])       
            
        df['lon'] = self.lat_lons[self.kd_tree.query(df[['longitude','latitude']],1,return_distance=False)[:,0]][:,0]
        df['lat'] = self.lat_lons[self.kd_tree.query(df[['longitude','latitude']],1,return_distance=False)[:,0]][:,1]
        df['time'] = df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
        
        for nearest in [2,3,4]:
            df['lon_near_{}'.format(nearest)] = self.lat_lons[self.kd_tree.query(df[['longitude','latitude']],4,return_distance=False)[:,nearest-1]][:,0]
            df['lat_near_{}'.format(nearest)] = self.lat_lons[self.kd_tree.query(df[['longitude','latitude']],4,return_distance=False)[:,nearest-1]][:,1]  
   
        features = self.make_raw_features(df)
        print(features.shape)
        features = features.merge(self.make_winds_features(df),left_index=True,right_index=True)
        print(features.shape)
        features = features.merge(self.make_temp_rhum_features(df),left_index=True,right_index=True)
        return features.drop(['lat_near_2','lat_near_3','lat_near_4',
                              'lon_near_2','lon_near_3','lon_near_4'],errors='ignore')
    
    def make_labels(self,df):
        return df.set_index('fire_id')['fire_type']
    
    def make_train_set(self,df):
        X = self.make_all_features(df)
        y = self.make_labels(df)
        return X,y
    
    
def time_series_features(full_df,train,test):
    '''
        full_df - весь датасет
        train - обучающая выборка
        test - тестовая выборка
        
        алгоритм такой:
          1) кластеризация делаем по всему full_df;
          2) количество, площади и т.д. считаем только по train
          3) получившиеся фичи из train энкодим в test по метке кластера
    '''
    dbs = DBSCAN(eps=0.035,min_samples=1,n_jobs=4)
    full_df.sort_index(inplace=True)
    train.sort_index(inplace=True)
    test.sort_index(inplace=True)
    full_df['label'] = dbs.fit_predict(X[['latitude','longitude']])
    train['label'] = full_df.loc[train.index,'label']
    test['label'] = full_df.loc[test.index,'label']
    train['count_035'] = train.groupby('label')['latitude'].transform('count')
    train['dist_035_latitude_mean'] = train.groupby('label')['latitude'].transform('mean')
    train['dist_035_longitude_mean'] = train.groupby('label')['longitude'].transform('mean')
    train['dist_035_square'] = (train.groupby('label')['latitude'].transform('max') - \
                                   train.groupby('label')['latitude'].transform('min'))*\
                                (train.groupby('label')['longitude'].transform('max') - \
                                   train.groupby('label')['longitude'].transform('min'))
    for col in ['dist_035_latitude_mean','dist_035_longitude_mean','dist_035_square','count_035']:
        map_dict = train.drop_duplicates('label').set_index('label')[col]
        test[col] = full_df.loc[test.index,'label'].map(map_dict)
        nan_ind = test[test[col].isnull()].index
        if col == 'count_035':
            test.loc[nan_ind,col] = 0
        test.loc[nan_ind,col] = -1

    train['distance_from_035'] = train[['dist_035_longitude_mean','dist_035_latitude_mean','longitude','latitude']]\
                                .apply(lambda x: calc_distance(x['dist_035_longitude_mean'],x['dist_035_latitude_mean'],x['longitude'],x['latitude']),axis=1)
    test['distance_from_035'] = test[['dist_035_longitude_mean','dist_035_latitude_mean','longitude','latitude']]\
                                .apply(lambda x: calc_distance(x['dist_035_longitude_mean'],x['dist_035_latitude_mean'],x['longitude'],x['latitude']),axis=1)
    
    dbs = DBSCAN(eps=0.15,min_samples=1,n_jobs=4)
    full_df['label'] = dbs.fit_predict(X[['latitude','longitude']])
    train['label'] = full_df.loc[train.index,'label']
    test['label'] = full_df.loc[test.index,'label']
    train['count_15'] = train.groupby('label')['latitude'].transform('count')
    train['dist_15_square'] = (train.groupby('label')['latitude'].transform('max') - \
                                   train.groupby('label')['latitude'].transform('min'))*\
                                (train.groupby('label')['longitude'].transform('max') - \
                                   train.groupby('label')['longitude'].transform('min'))
    for col in ['dist_15_square','count_15']:
        map_dict = train.drop_duplicates('label').set_index('label')[col]
        test[col] = full_df.loc[test.index,'label'].map(map_dict)
        nan_ind = test[test[col].isnull()].index
        if col == 'count_15':
            test.loc[nan_ind,col] = 0
        test.loc[nan_ind,col] = -1
    train['square_ratio'] = (train['dist_035_square']/train['dist_15_square']).fillna(0)
    test['square_ratio'] = (test['dist_035_square']/test['dist_15_square']).fillna(0)
    train['count_035_divide_count_15'] = (train['count_035']/train['count_15']).fillna(0)
    test['count_035_divide_count_15'] = (test['count_035']/test['count_15']).fillna(0)

    #train['latitude_big_round'] = train['latitude'].round(0)
    #train['longitude_big_round'] = train['longitude'].round(0)
    #test['latitude_big_round'] = test['latitude'].round(0)
    #test['longitude_big_round'] = test['longitude'].round(0)
    return pd.concat([train,test]).drop('label',axis=1)

In [15]:
kd_tree = KDTree(df_train[['longitude','latitude']])
kd_time_tree = KDTree(df_train[['longitude','latitude','timestamp']])

date_list = []
start_date = datetime.date(2011,10,1)
while start_date < datetime.date(2019,5,1):
    date_list.append(start_date)
    start_date += relativedelta(days=1)
min_date_dict = pd.Series(np.nan,index=pd.to_datetime(date_list))
max_date_dict = min_date_dict.copy()
min_date_dict = df_train.drop_duplicates('date',keep='first').reset_index().set_index('date')['fire_id'].\
                       combine(min_date_dict,max).fillna(method='bfill').fillna(-1).astype(int).to_dict()
max_date_dict = df_train.drop_duplicates('date',keep='first').reset_index().set_index('date')['fire_id'].\
                       combine(max_date_dict,max).fillna(method='ffill').fillna(-1).astype(int).to_dict()
def return_nearest_previous_coord(row):
    indices,distances = kd_tree.query_radius([[row['longitude'],row['latitude']]],0.05,
                                    return_distance=True,sort_results=True)
    indices = indices[0][1:]
   #print(indices)
    
    count_3_years = 0
    
    count_last_year = 0
    
    lat_last_year_nearest = 0
    lon_last_year_nearest = 0
    distance_last_year_nearest = -1
    bearing_last_year_nearest = -200
    last_year_class = -1
    
    delta_week = relativedelta(days=7)
    delta_month = relativedelta(days=14)
    delta_year = relativedelta(years=1)
    start_year = row['date'] - delta_year
    condition = None
    condition_month = None
    condition_week = None
    last_year_condition = None
    last_year_condition_month = None
    end_year = row['date'].replace(year=2019,day=1,month=1)
    n = 0
    #print(start_year)
    while start_year >= datetime.date(2012,1,1):
        if n >= 3:
            break
        min_date_month = start_year - delta_month
        max_date_month = start_year + delta_month  
        min_date_week = start_year - delta_week
        max_date_week = start_year + delta_week   
        try:
            if condition is None:
                #condition = ((indices <= max_date_dict[max_date]) & (indices >= min_date_dict[min_date]))
                condition_month = ((indices <= max_date_dict[max_date_month]) & (indices >= min_date_dict[min_date_month]))
                condition_week = ((indices <= max_date_dict[max_date_week]) & (indices >= min_date_dict[min_date_week]))
                #last_year_condition = (indices <= max_date_dict[max_date]) & (indices >= min_date_dict[min_date])
                last_year_condition_month = (indices <= max_date_dict[max_date_month]) & (indices >= min_date_dict[min_date_month])
            else:
                #condition |= ((indices <= max_date_dict[max_date]) & (indices >= min_date_dict[min_date]))
                condition_month = ((indices <= max_date_dict[max_date_month]) & (indices >= min_date_dict[min_date_month]))
                condition_week = ((indices <= max_date_dict[max_date_week]) & (indices >= min_date_dict[min_date_week]))
        except:
            print(max_date,min_date)
            #print(condition)
            raise
        start_year -= delta_year
        n += 1
    #print(condition)
    #print(last_year_condition)
    #filtered_ind = np.where(last_year_condition)[0]
    #good_ind = indices[filtered_ind]
    #merge_dict = {'full_count_5km':len(indices)}
    if len(indices) > 0:
        all_condition = (indices <= max_date_dict[end_year]) 
        filtered_ind = np.where(all_condition)[0]
        merge_dict = {'5km_count':len(filtered_ind)}
    else:
        merge_dict = {'5km_count':0}
        
    filtered_ind = np.where(last_year_condition_month)[0]
    good_ind = indices[filtered_ind]
    merge_dict.update({'last_3_years_count':len(good_ind)})
    if len(good_ind) > 0:
        lat_last_year_nearest = df_train.iloc[good_ind[0]]['latitude']
        lon_last_year_nearest = df_train.iloc[good_ind[0]]['longitude']
        distance_last_year_nearest = calc_distance(row['longitude'],row['latitude'],
                                                   lon_last_year_nearest,lat_last_year_nearest)
        bearing_last_year_nearest = calc_bearing(row['longitude'],row['latitude'],
                                                  lon_last_year_nearest,lat_last_year_nearest)
        
    merge_dict.update({'diff_month_nearest_latitude':lat_last_year_nearest,
                      'diff_month_nearest_longitude':lon_last_year_nearest,
                      'diff_month_nearest_distance':distance_last_year_nearest,
                      'diff_month_count':len(good_ind),
                     })
    
    
    filtered_ind = np.where(condition_month)[0]
    good_ind = indices[filtered_ind]
    merge_dict.update({'diff_3_years_month_count':len(good_ind)})
    if len(good_ind) > 0:
        lat_last_year_nearest = df_train.iloc[good_ind[0]]['latitude']
        lon_last_year_nearest = df_train.iloc[good_ind[0]]['longitude']
        distance_last_year_nearest = calc_distance(row['longitude'],row['latitude'],
                                                   lon_last_year_nearest,lat_last_year_nearest)
        bearing_last_year_nearest = calc_bearing(row['longitude'],row['latitude'],
                                                  lon_last_year_nearest,lat_last_year_nearest)
        
    merge_dict.update({'diff_3_years_month_nearest_latitude':lat_last_year_nearest,
                      'diff_3_years_month_nearest_longitude':lon_last_year_nearest,
                      'diff_3_years_month_nearest_distance':distance_last_year_nearest,
                      'diff_3_years_month_count':len(good_ind),
                     })
    
    filtered_ind = np.where(condition_week)[0]
    good_ind = indices[filtered_ind]
    merge_dict.update({'last_year_diff_week_count':len(good_ind)})
    
    indices,distances = kd_time_tree.query_radius([[row['longitude'],row['latitude'],row['timestamp']]],
                            0.1,return_distance=True,sort_results=True)
    indices = indices[0][1:]
    
    count_same_day = 0
    lat_same_day_nearest = 0
    lon_same_day_nearest = 0
    distance_same_day_nearest = -1
    bearing_same_day_nearest = -200   
    if len(indices) > 0:
        lat_same_day_nearest = df_train.iloc[indices[0]]['latitude']
        lon_same_day_nearest = df_train.iloc[indices[0]]['longitude']
        distance_same_day_nearest = calc_distance(row['longitude'],row['latitude'],
                                                   lon_same_day_nearest,lat_same_day_nearest)
        bearing_same_day_nearest = calc_bearing(row['longitude'],row['latitude'],
                                                  lon_same_day_nearest,lat_same_day_nearest)
        count_same_day = len(indices)
    merge_dict.update({'same_day_nearest_latitude_10km':lat_same_day_nearest,
                      'same_day_nearest_longitude_10km':lon_same_day_nearest,
                      'same_day_nearest_distance_10km':distance_same_day_nearest,
                      'same_day_nearest_bearing_10km':bearing_same_day_nearest,
                      'same_day_count_10km':count_same_day
                     }) 
    
    indices,distances = kd_time_tree.query_radius([[row['longitude'],row['latitude'],row['timestamp']]],
                            0.25,return_distance=True,sort_results=True)
    indices = indices[0][1:]
    
    count_same_day = 0
    lat_same_day_nearest = 0
    lon_same_day_nearest = 0
    distance_same_day_nearest = -1
    bearing_same_day_nearest = -200   
    if len(indices) > 0:
        lat_same_day_nearest = df_train.iloc[indices[0]]['latitude']
        lon_same_day_nearest = df_train.iloc[indices[0]]['longitude']
        distance_same_day_nearest = calc_distance(row['longitude'],row['latitude'],
                                                   lon_same_day_nearest,lat_same_day_nearest)
        bearing_same_day_nearest = calc_bearing(row['longitude'],row['latitude'],
                                                  lon_same_day_nearest,lat_same_day_nearest)
        count_same_day = len(indices)
    merge_dict.update({'same_day_nearest_latitude_25km':lat_same_day_nearest,
                      'same_day_nearest_longitude_25km':lon_same_day_nearest,
                      'same_day_nearest_distance_25km':distance_same_day_nearest,
                      'same_day_nearest_bearing_25km':bearing_same_day_nearest,
                      'same_day_count_25km':count_same_day
                     })
    
    indices,distances = kd_tree.query_radius([[row['longitude'],row['latitude']]],0.2,
                                    return_distance=True,sort_results=True)
    indices = indices[0][1:]
    if len(indices) > 0:
        all_condition = (indices <= max_date_dict[end_year])  
        filtered_ind = np.where(all_condition)[0]
        merge_dict.update({'20km_count':len(filtered_ind)})
        if merge_dict['20km_count'] > 0:
            merge_dict['5km_divide_20km'] = merge_dict['5km_count']/merge_dict['20km_count']
        else:
            merge_dict['5km_divide_20km'] = -1
        #except:
            #print(len(indices),indices,max_date_dict[end_year],len(filtered_ind),merge_dict['20km_count'],
                # merge_dict['5km_count'])
            #raise
    else:
        merge_dict.update({'20km_count':0})
        merge_dict['5km_divide_20km'] = -1
    
    return pd.Series(merge_dict)

In [10]:
fm = FeaturesMaker()
features_df = fm.make_all_features(df_train.reset_index())

Making raw features...
Making raw features done! Time 2.26
(174871, 6)
Reading winds dataset...
uwnd
vwnd
Reading dataset done! Time 72.68
Making wind features...
2019-11-13 00:00:00 2012-01-01 00:00:00
Making wind features done! Time 1961.27
(174871, 308)
(174871, 313)
Reading temperature-rhum-dataset...
rhum
air
Reading dataset done! Time 65.84
Making temperature-humidity features...
2019-11-13 00:00:00 2012-01-01 00:00:00
(174871, 540)
Making temperature-humidity features done! Time 2034.18


In [11]:
features_df.shape

(174871, 852)

In [16]:
%time raw_features_df = df_train[['latitude','longitude','date','timestamp']].apply(return_nearest_previous_coord,axis=1)

Wall time: 6min 16s


In [17]:
X = pd.merge(features_df,raw_features_df,left_index=True,right_index=True)
print(X.shape)
X.drop(X.count()[X.count() <= 0].index,axis=1,inplace=True)
X.replace({np.inf:np.nan,-np.inf:np.nan},inplace=True)
X.fillna(method='ffill',inplace=True)
X.fillna(method='bfill',inplace=True)
print(X.shape)

y = df_train['fire_type']
print(y.shape)

(174871, 875)
(174871, 727)
(174871,)


In [18]:
# тут фичи, которые получаются путем агрегации разных фич по координатам сетки ncep.
# transform_dict я потом задампил, чтобы сохранить и использовать в скрипте
# это последние закоменченные строчки 

var_dict = {'t':'temperature','h':'humidity','uw':'uwind'}

transform_dict = {}

X['lat-lon'] = X['lat'].astype(str) + '-' + X['lon'].astype(str)
for level in fm.base_levels:
    print(level)
    for var in ['t','h','uw']:
        try:
            col = 'p14d_diff-{}_level{}_latlon'.format(var,level)
            X[col]= X.groupby(['lat','lon'])['p14d_diff-{}_level{}'.format(var,level)].transform('mean')
            transform_dict.update({col:X.drop_duplicates('lat-lon',keep='first')\
                                       .set_index('lat-lon')[col].to_dict()})
        except:
            pass
        try:
            col = 'diff_first_mean-{}_level{}_latlon'.format(var,level)
            X[col] = X.groupby(['lat','lon'])['diff_first_mean-{}_level{}'.format(var,level)].transform('mean')  
            transform_dict.update({col:X.drop_duplicates('lat-lon',keep='first')\
                                       .set_index('lat-lon')[col].to_dict()})
        except:
            pass
        new_var = var_dict[var]
        try:
            col = '{}_level{}_latlon'.format(new_var,level)
            X[col] = X.groupby(['lat','lon'])['{}_level{}'.format(new_var,level)].transform('mean')  
            transform_dict.update({col:X.drop_duplicates('lat-lon',keep='first')\
                                       .set_index('lat-lon')[col].to_dict()})
        except:
            pass

'''
import json
import io

with io.open('transform_dict-2.json','w',encoding='utf-8') as f:
    f.write(json.dumps(transform_dict))     
'''

1000
700
500
300
250
200
100
150
70
50


"\nimport json\nimport io\n\nwith io.open('transform_dict-2.json','w',encoding='utf-8') as f:\n    f.write(json.dumps(transform_dict))     \n"

In [19]:
print(X.shape)

(174871, 800)


In [20]:
# ------------------------------
# фичи с городами
biggest_cities = pd.read_csv('biggest_cities.csv')
biggest_cities.loc[923,'Население'] = 400
biggest_cities['Население'] = biggest_cities['Население'].astype(np.int)
for _ind in [506,782,863]:
    biggest_cities.loc[_ind,'Город'] = biggest_cities.loc[_ind,'Регион']
biggest_cities.loc[1066,'Город'] = 'Урус-Мартан'
biggest_cities = biggest_cities[['Широта','Долгота','Население','Город']]

kd_tree_cities = KDTree(biggest_cities[['Долгота','Широта']])
X['city_lon'] = biggest_cities.iloc[kd_tree_cities.query(X[['longitude','latitude']],1,return_distance=False)[:,0]]['Долгота'].values
X['city_lat'] = biggest_cities.iloc[kd_tree_cities.query(X[['longitude','latitude']],1,return_distance=False)[:,0]]['Широта'].values
X['city_population'] = biggest_cities.iloc[kd_tree_cities.query(X[['longitude','latitude']],1,return_distance=False)[:,0]]['Население'].values
X['distance_from_city'] = X[['city_lon','city_lat','latitude','longitude']]\
                           .apply(lambda x: calc_distance(x['city_lon'],x['city_lat'],
                                                x['longitude'],x['latitude']),axis=1)

# ------------------------------------------
# фичи с полями, лесами и заповедниками
for _file in ['field_coords','forest_coords','nature_forests']:
    forest_df = pd.read_csv('{}.csv'.format(_file))
    kd_tree_nature = KDTree(forest_df[['longitude','latitude']])
    X['{}_lon'.format(_file)] = forest_df.iloc[kd_tree_nature.query(X[['longitude','latitude']],1,return_distance=False)[:,0]]['longitude'].values
    X['{}_lat'.format(_file)] = forest_df.iloc[kd_tree_nature.query(X[['longitude','latitude']],1,return_distance=False)[:,0]]['latitude'].values
    X['distance_from_{}'.format(_file)] = X[['{}_lon'.format(_file),'{}_lat'.format(_file),'latitude','longitude']]\
                           .apply(lambda x: calc_distance(x['{}_lon'.format(_file)],x['{}_lat'.format(_file)],
                                               x['longitude'],x['latitude']),axis=1)
    
print(X.shape)

(174871, 813)


# Ниже два метода для кросс-валидации

У каждого из них одинаковый подход к кросс-валидации:
* задается число фолдов n
* для k от 0 до n-1:
* из выборки удаляются последние k месяцев
* тест - последние 4 месяца из оставшейся выборки, трейн - все остальное

Кросс-валидация по времени нужна, чтобы не оверфитнуться на какой-то конкретный период - решение сабмитил только в том случае, когда улучшение было только на всех фолдах (кроме преодоления рубежа 0.9297, до 0.9305 частично дошел пробингом, там на некоторых фолдах было улучшение, на некоторых нет)

Первый метод - обычная кросс-валидация, нужна для проверки качества

Второй метод - кросс-валидация с отбором фич по Permutation Importance. Почему-то eli5 работал дольше, чем написанный вручную и кстати написал я его не совсем правильно :) Моя реализация:
* вначале считаем качество (roc-auc) на тестовой выборке
* для каждой фичи проделываем следующую процедуру несколько раз (в моей реализации 5):
<ol>
    <li> перемешиваем в тесте значения этой фичи, повторно вычисляем roc-auc с перемешанной фичей; </li>
    <li> считаем разницу между исходным auc и auc с перемешанной фичей. </li>
    <li> чем больше эта разница, тем важнее фича (по крайней мере для метрики auc). </li>
</ol>

Почему у меня неправильная реализация - в Permutation Importance нужно вычислить распределение конкретного признака и случайным образом из этого распределения заполнить значения в тесте. Чтобы узнать распределение нужно использовать весь датасет (трейн+тест), а я тупо перемешивал значения из теста. Таким образом, например, по фиче "year" разница между ихсодным auc и перемешанным всегда 0, т.к. при моей реализации кросс-валидации в тесте год всегда единственное значение и перемешивание ничего не меняло.

За счет отбора фич по permutation importance я смог прыгнуть с 0.9253 до 0.9289

***Permutation Importance работает довольно долго (у меня 2.5 дня), поэтому сюда это я включать не буду. Можно запустить самому***

<code>cross_val_permutation_importance(X,y-1,df_train,folds=4,cat_features=[],features_list=list(X.columns))</code>

In [21]:
lb = LabelBinarizer()
lb.fit(df_train['fire_type']-1)

def lgb_auc(preds,train):
    '''
        кастомная метрика для micro roc-auc для lightgbm
        preds - np.array, предсказания от lightgbm
        train - lgb.Dataset, обучающая выборка
    '''
    y = train.get_label()
    a = preds.reshape(11,-1).T
    return 'roc_auc',roc_auc_score(lb.transform(y),a,average='micro'),True

In [22]:
def cross_val(X,y,df_dates,folds=3,features_list=None,cat_features=None,
                           params_lgb=None,file_name='feature-importances-cv.csv'):
    '''
        стандартная кросс-валидация
        
        X - признаки
        y - лейблы
        df_dates - исходный датасет (нужен чтобы на каждом фолде правильно 
                                     заполнить временные фичи за прошлый год 
                                    + для разбиения фолдов по дате)
        features_list - список используемых фичей
        cat_features - список фичей, который будут категориальными
        params_lgb - параметры для lightgbm
        file_name - имя файла, куда будет сохранено feature importance (от lightgbm, importance по количеству разбиений)
    '''
    if cat_features is None:
        cat_features=['month','year']
    #df_dates['date'] = pd.to_datetime(df_dates['date'])
    df_dates.sort_values('date',inplace=True)
    feature_importances = pd.DataFrame(columns=['fold','date-diapason','feature','importance'])
    dif_months = 4
    for fold in range(folds):
        #if fold <= 3 or fold in [5,6]:
         #   continue
        #if fold in [1,2]:
        max_date = datetime.date(2019,5,1) - relativedelta(months=dif_months*(fold))
        #else:
        #max_date = datetime.date(2019,5,1)
        last_date = max_date - relativedelta(months=dif_months)
        print(datetime.date(last_date.year,1,1))
        prev_year_ind = df_dates[df_dates['date'] < datetime.date(last_date.year,1,1)].index
        now_ind = df_dates[df_dates['date'] >= datetime.date(last_date.year,1,1)].index
        print(X.shape)
        X_feat = time_series_features(df_dates,X.loc[prev_year_ind],X.loc[now_ind])
        print(X_feat.shape)
        print(list(X_feat.columns.difference(X.columns)))
        print(max_date,last_date)

        train_ind = df_dates[df_dates['date'] < last_date].index
        test_ind = df_dates[(df_dates['date'] >= last_date) & (df_dates['date'] < max_date)].index
        print('Train length {}.  Test length {}'.format(len(train_ind),len(test_ind)))
        X_train, y_train = X_feat.loc[train_ind],y.loc[train_ind]
        X_test, y_test = X_feat.loc[test_ind],y.loc[test_ind]
        
        if params_lgb is None:
            params_lgb = {
                'bagging_freq': 5,
                'bagging_fraction': 0.42,
                'feature_fraction':0.4,
                'bagging_seed':400,
                'feature_fraction_seed':400,
                'boost': 'gbdt',
                'learning_rate': 0.008,
                'min_data_in_leaf':4,
                'num_leaves': 13,
                'num_threads': 6,
                'tree_learner': 'voting',
                'objective': 'multiclass',
                'verbosity': 1,
                'lambda_l2':2,
                'num_classes': 11,
                'seed':4567
            }
        print(params_lgb)

        print(len(features_list))
        train_data = lgb.Dataset(X_train[features_list], label=y_train.values)
        test_data = lgb.Dataset(X_test[features_list], label=y_test.values)
        
        # !!!!!!!!!!!!!!!!!!!!!!
        # я здесь убрал метрики для train, т.к. с ними намного дольше
        # а по качеству ни разу не было останова из-за трейна
        lgb_model = lgb.train(params_lgb,train_data,num_boost_round=20000,
                            valid_sets = [test_data],verbose_eval=100,early_stopping_rounds = 400,
                                  feature_name = features_list,feval=lgb_auc,
                                 categorical_feature = cat_features) 
        lgb_model.save_model('best-features-{}-fold-{}.dmp'.format(len(features_list),fold))
        part_feature_importances = pd.DataFrame(columns=['fold','date-diapason','feature','importance'])
        n = 0
        for feature_name,importance in zip(lgb_model.feature_name(),lgb_model.feature_importance()):
            part_feature_importances.loc[n] = [fold + 1,'{}-{}'.format(last_date.strftime('%Y-%m-%d'),
                                        max_date.strftime('%Y-%m-%d')),feature_name,int(importance)]
            n += 1
        feature_importances = pd.concat([feature_importances,part_feature_importances],ignore_index=True,axis=0)
        feature_importances.to_csv(file_name)
    return feature_importances


def cross_val_permutation_importance(X,y,df_dates,folds=3,features_list=None,
                   cat_features=None,file_name='permutation-feature-importances.csv',
                                    permutation_iters=5):
    '''
        кросс-валидация с вычислением permutation importance
        
        X - признаки
        y - лейблы
        df_dates - исходный датасет (нужен чтобы на каждом фолде правильно 
                                     заполнить временные фичи за прошлый год 
                                    + для разбиения фолдов по дате)
        features_list - список используемых фичей
        cat_features - список фичей, который будут категориальными
        params_lgb - параметры для lightgbm
        file_name - имя файла, куда будет сохранено feature importance 
                               (от lightgbm, importance по количеству разбиений +
                               permutation importance)
        permutation_iters - количество итераций при перемешивании признака в permutation importance
    '''
    if cat_features is None:
        cat_features=['month','year']
    #df_dates['date'] = pd.to_datetime(df_dates['date'])
    df_dates.sort_values('date',inplace=True)
    feature_importances_df = pd.DataFrame(columns=['fold','date-diapason','feature','importance'])
    dif_months = 4
    
    for fold in range(folds):
        if fold <= 1:
            continue
        #if fold in [1,2]:
        max_date = datetime.date(2019,5,1) - relativedelta(months=dif_months*(fold))
        #else:
        #max_date = datetime.date(2019,5,1)
        last_date = max_date - relativedelta(months=dif_months)
        print(datetime.date(last_date.year,1,1))
        prev_year_ind = df_dates[df_dates['date'] < datetime.date(last_date.year,1,1)].index
        now_ind = df_dates[df_dates['date'] >= datetime.date(last_date.year,1,1)].index
        print(X.shape)
        X_feat = time_series_features(df_dates,X.loc[prev_year_ind],X.loc[now_ind])
        print(X_feat.shape)
        print(list(X_feat.columns.difference(X.columns)))
        print(max_date,last_date)

        train_ind = df_dates[df_dates['date'] < last_date].index
        test_ind = df_dates[(df_dates['date'] >= last_date) & (df_dates['date'] < max_date)].index
        print('Train length {}.  Test length {}'.format(len(train_ind),len(test_ind)))
        X_train, y_train = X_feat.loc[train_ind],y.loc[train_ind]
        X_test, y_test = X_feat.loc[test_ind],y.loc[test_ind]
        
        params_lgb = {
            'bagging_freq': 5,
            'bagging_fraction': 0.41,
            'feature_fraction':0.4,
            'bagging_seed':400,
            'feature_fraction_seed':400,
            'boost': 'gbdt',
            'learning_rate': 0.008,
            'min_data_in_leaf':4,
            'num_leaves': 13,
            'num_threads': 6,
            'tree_learner': 'voting',
            'objective': 'multiclass',
            'verbosity': 1,
            'lambda_l2':2,
            'num_classes': 11,
            'seed':4567
        }

        feature_names = list(X_feat.columns)
        train_data = lgb.Dataset(X_train[features_list], label=y_train.values)
        print('features count: {}'.format(len(train_data.feature_name)))
        test_data = lgb.Dataset(X_test[features_list], label=y_test.values)
        
        # !!!!!!!!!!!!!!!!!!!!!!
        # я здесь убрал метрики для train, т.к. с ними намного дольше
        # а по качеству ни разу не было останова из-за трейна
        lgb_model = lgb.train(params_lgb,train_data,num_boost_round=10000,
                            valid_sets = [test_data],verbose_eval=100,early_stopping_rounds = 400,
                                  feature_name =features_list,feval=lgb_auc,
                                 categorical_feature = cat_features)
        
        part_feature_importances = pd.DataFrame(columns=['fold','date-diapason','iter','feature',
                                                         'permutation-importance','importance'])
        n = 0
        start = time.time()
        good_preds = lgb_model.predict(X_test[feature_names])
        good_auc = roc_auc_score(lb.transform(y_test),good_preds,average='micro')
        for feature_name,imp in zip(lgb_model.feature_name(),lgb_model.feature_importance()):
            a = X_test[feature_name].copy()
            for _iter in range(4):
                X_test[feature_name] = X_test[feature_name].sample(frac=1).values
                preds = lgb_model.predict(X_test[feature_names])
                permutation_auc = roc_auc_score(lb.transform(y_test),preds,average='micro')
                part_feature_importances.loc[n] = [fold + 1,'{}-{}'.format(last_date.strftime('%Y-%m-%d'),
                                            max_date.strftime('%Y-%m-%d')),_iter,feature_name,
                                                   good_auc-permutation_auc,imp]
                n += 1
                #part_feature_importances.to_csv(file_name)
            X_test[feature_name] = a.copy()
        end = time.time()
        print(end-start)
        feature_importances_df = pd.concat([feature_importances_df,part_feature_importances],ignore_index=True,axis=0)
        feature_importances_df.to_csv(file_name)
    return feature_importances

In [26]:
# это уже кросс-валидация с отобранными фичами
cross_val(X,y-1,df_train,folds=4,cat_features=[],features_list=model_features)

2019-01-01
(174871, 813)
(174871, 822)
['count_035', 'count_035_divide_count_15', 'count_15', 'dist_035_latitude_mean', 'dist_035_longitude_mean', 'dist_035_square', 'dist_15_square', 'distance_from_035', 'square_ratio']
2019-05-01 2019-01-01
Train length 158860.  Test length 16011
{'bagging_freq': 5, 'bagging_fraction': 0.42, 'feature_fraction': 0.4, 'bagging_seed': 400, 'feature_fraction_seed': 400, 'boost': 'gbdt', 'learning_rate': 0.008, 'min_data_in_leaf': 4, 'num_leaves': 13, 'num_threads': 6, 'tree_learner': 'voting', 'objective': 'multiclass', 'verbosity': 1, 'lambda_l2': 2, 'num_classes': 11, 'seed': 4567}
214
Training until validation scores don't improve for 400 rounds
[100]	valid_0's multi_logloss: 1.4903	valid_0's roc_auc: 0.906967
[200]	valid_0's multi_logloss: 1.36509	valid_0's roc_auc: 0.917348
[300]	valid_0's multi_logloss: 1.30469	valid_0's roc_auc: 0.920515
[400]	valid_0's multi_logloss: 1.26994	valid_0's roc_auc: 0.922576
[500]	valid_0's multi_logloss: 1.24978	valid

2018-01-01
(174871, 813)
(174871, 822)
['count_035', 'count_035_divide_count_15', 'count_15', 'dist_035_latitude_mean', 'dist_035_longitude_mean', 'dist_035_square', 'dist_15_square', 'distance_from_035', 'square_ratio']
2018-09-01 2018-05-01
Train length 145944.  Test length 6465
{'bagging_freq': 5, 'bagging_fraction': 0.42, 'feature_fraction': 0.4, 'bagging_seed': 400, 'feature_fraction_seed': 400, 'boost': 'gbdt', 'learning_rate': 0.008, 'min_data_in_leaf': 4, 'num_leaves': 13, 'num_threads': 6, 'tree_learner': 'voting', 'objective': 'multiclass', 'verbosity': 1, 'lambda_l2': 2, 'num_classes': 11, 'seed': 4567}
214
Training until validation scores don't improve for 400 rounds
[100]	valid_0's multi_logloss: 1.5392	valid_0's roc_auc: 0.894587
[200]	valid_0's multi_logloss: 1.34917	valid_0's roc_auc: 0.920765
[300]	valid_0's multi_logloss: 1.24701	valid_0's roc_auc: 0.929391
[400]	valid_0's multi_logloss: 1.18258	valid_0's roc_auc: 0.934867
[500]	valid_0's multi_logloss: 1.14229	valid_

[1500]	valid_0's multi_logloss: 1.26926	valid_0's roc_auc: 0.919825
[1600]	valid_0's multi_logloss: 1.27012	valid_0's roc_auc: 0.919757
[1700]	valid_0's multi_logloss: 1.27063	valid_0's roc_auc: 0.919757
[1800]	valid_0's multi_logloss: 1.27024	valid_0's roc_auc: 0.919892
[1900]	valid_0's multi_logloss: 1.26989	valid_0's roc_auc: 0.919977
Early stopping, best iteration is:
[1519]	valid_0's multi_logloss: 1.2688	valid_0's roc_auc: 0.91988


,fold,date-diapason,feature,importance
0,1,2019-01-01-2019-05-01,p14d-uw_level_100_50_diff,6246
1,1,2019-01-01-2019-05-01,first_uwind_near_2_level_200,1966
2,1,2019-01-01-2019-05-01,humidity_level300_latlon,1900
3,1,2019-01-01-2019-05-01,first_uwind_near_3_level_300,2013
4,1,2019-01-01-2019-05-01,uwind_level100_latlon,1159
...,...,...,...,...
851,4,2018-01-01-2018-05-01,forest_coords_lon,889
852,4,2018-01-01-2018-05-01,distance_from_forest_coords,1414
853,4,2018-01-01-2018-05-01,field_coords_lat,756
854,4,2018-01-01-2018-05-01,field_coords_lon,1104


# Сохранение модели

In [ ]:
params_lgb={
                'bagging_freq': 5,
                'bagging_fraction': 0.42,
                'feature_fraction':0.4,
                'bagging_seed':400,
                'feature_fraction_seed':400,
                'boost': 'gbdt',
                'learning_rate': 0.008,
                'min_data_in_leaf':4,
                'num_leaves': 13,
                'num_threads': 8,
                'tree_learner': 'voting',
                'objective': 'multiclass',
                'verbosity': 1,
                'lambda_l2':2,
                'num_classes': 11,
                'seed':4567
            }

prev_year_ind = df_train[df_train['date'] < datetime.date(2019,1,1)].index
now_ind = df_train[df_train['date'] >= datetime.date(2019,1,1)].index
print(X.shape)
X_feat = time_series_features(df_train,X.loc[prev_year_ind],X.loc[now_ind])
y = df_train.loc[X_feat.index,'fire_type']
train_data = lgb.Dataset(X_feat[model_features], label=y-1,
                        feature_name=model_features,
                        categorical_feature = [])
print(X_feat[features].shape)
print('number of features {}'.format(len(train_data.feature_name)))
print('categorical features {}'.format(train_data.categorical_feature))

lgb_model = lgb.train(params_lgb,train_data,num_boost_round=3200,feature_name=model_features) 

lgb_model.save_model('lgbm_model_3200_iter_{}_filtered_features-without_month.dmp'.format(len(model_features)))